## Preprocessing

In [80]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import joblib

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()
application_df.columns

Index(['EIN', 'NAME', 'APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION',
       'USE_CASE', 'ORGANIZATION', 'STATUS', 'INCOME_AMT',
       'SPECIAL_CONSIDERATIONS', 'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

In [64]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN","NAME"], axis = 1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [65]:
# Determine the number of unique values in each column.
print(application_df.nunique())

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [66]:
# Look at APPLICATION_TYPE value counts for binning
print(application_df["APPLICATION_TYPE"].value_counts())

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [67]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff = 500
value_counts = application_df["APPLICATION_TYPE"].value_counts()
application_types_to_replace = value_counts.index[value_counts < cutoff].tolist()
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [68]:
# Look at CLASSIFICATION value counts for binning
print(application_df["CLASSIFICATION"].value_counts())

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [69]:
# You may find it helpful to look at CLASSIFICATION value counts >1
value_counts = application_df["CLASSIFICATION"].value_counts()
value_countsgreater = value_counts[value_counts > 1]
print(value_countsgreater)


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64


In [70]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

cutoff = 1000
classifications_to_replace = value_counts.index[value_counts < cutoff].tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [71]:
# Convert categorical data to numeric with `pd.get_dummies`
df_dummies = pd.get_dummies(application_df, columns=['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT'])
df_dummies.head()

,STATUS,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,N,5000,1,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,N,108590,1,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,N,5000,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
3,1,N,6692,1,0,0,0,1,0,0,...,1,0,0,1,0,0,0,0,0,0
4,1,N,142590,1,0,0,0,1,0,0,...,1,0,0,0,1,0,0,0,0,0


In [72]:
# Split our preprocessed data into our features and target arrays
x_data = df_dummies.drop(columns = ['IS_SUCCESSFUL', 'STATUS', 'SPECIAL_CONSIDERATIONS'])
X = x_data
y = df_dummies["IS_SUCCESSFUL"]
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)

In [73]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [74]:
print(len(X_train_scaled[0]))

40


## Compile, Train and Evaluate the Model

In [75]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  90
hidden_nodes_layer2 = 45
hidden_nodes_layer3 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 90)                3690      
                                                                 
 dense_12 (Dense)            (None, 45)                4095      
                                                                 
 dense_13 (Dense)            (None, 30)                1380      
                                                                 
 dense_14 (Dense)            (None, 1)                 31        
                                                                 
Total params: 9,196
Trainable params: 9,196
Non-trainable params: 0
_________________________________________________________________


In [76]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [77]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=47)

Epoch 1/47
804/804 [==============================] - 4s 3ms/step - loss: 0.5669 - accuracy: 0.7233
Epoch 2/47
804/804 [==============================] - 2s 3ms/step - loss: 0.5540 - accuracy: 0.7283
Epoch 3/47
804/804 [==============================] - 2s 3ms/step - loss: 0.5509 - accuracy: 0.7306
Epoch 4/47
804/804 [==============================] - 3s 4ms/step - loss: 0.5486 - accuracy: 0.7311
Epoch 5/47
804/804 [==============================] - 3s 4ms/step - loss: 0.5478 - accuracy: 0.7320
Epoch 6/47
804/804 [==============================] - 3s 3ms/step - loss: 0.5466 - accuracy: 0.7324
Epoch 7/47
804/804 [==============================] - 3s 3ms/step - loss: 0.5457 - accuracy: 0.7338
Epoch 8/47
804/804 [==============================] - 3s 3ms/step - loss: 0.5451 - accuracy: 0.7343
Epoch 9/47
804/804 [==============================] - 4s 5ms/step - loss: 0.5445 - accuracy: 0.7341
Epoch 10/47
804/804 [==============================] - 3s 4ms/step - loss: 0.5443 - accuracy: 0.7346

In [78]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5594 - accuracy: 0.7319 - 614ms/epoch - 2ms/step
Loss: 0.5594344735145569, Accuracy: 0.7318950295448303


In [81]:
# Export our model to HDF5 file
joblib.dump(nn, 'trial2.h5')

['trial2.h5']